In [1]:
# import 
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torchvision import datasets, models, transforms
from utils import train, test, get_classification_report
import os

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
# data directory named 'moire'
data_dir = 'moire'
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

# create transform pipeline. This transformation will be applied on input image
# for data in train folder, RandomHorizontalFlip is added on top of resizing images to size 224 and then transform it into Tensor.
data_transforms = {
    TRAIN: transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}
# create dataset dictionary object
image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}
# create dataloaders object that shuffles the folders and creates batch of 32 images 
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=32,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)
print(image_datasets[VAL].classes)
print(image_datasets[TEST].classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Loaded 7268 images under train
Loaded 704 images under val
Loaded 665 images under test
Classes: 
['0', '1']
['0', '1']
['0', '1']


In [6]:
## Load the model based on VGG19
model = models.vgg19(pretrained=True)

# Modify the last layer having input size of number_features and output size of 2 (as we classify only 2 classes)
number_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1] # Remove last layer
features.extend([torch.nn.Linear(number_features, len(class_names))])
model.classifier = torch.nn.Sequential(*features)
model = model.to(device)

In [7]:
# Create criterion 
criterion = nn.CrossEntropyLoss()

# Create optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [8]:
# Train model, starting with the learned parameters from the initial VGG-19 model
model_trained = train(model,dataloaders, device, dataset_sizes,  criterion, optimizer, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.2877 Acc: 0.8690
val Loss: 0.1485 Acc: 0.9446

Epoch 1/24
----------
train Loss: 0.1678 Acc: 0.9305
val Loss: 0.2847 Acc: 0.9077

Epoch 2/24
----------
train Loss: 0.1253 Acc: 0.9468
val Loss: 0.1096 Acc: 0.9716

Epoch 3/24
----------
train Loss: 0.1407 Acc: 0.9345
val Loss: 0.0915 Acc: 0.9474

Epoch 4/24
----------
train Loss: 0.0572 Acc: 0.9809
val Loss: 0.0509 Acc: 0.9872

Epoch 5/24
----------
train Loss: 0.0513 Acc: 0.9816
val Loss: 0.0302 Acc: 0.9901

Epoch 6/24
----------
train Loss: 0.0594 Acc: 0.9789
val Loss: 0.0271 Acc: 0.9915

Epoch 7/24
----------
train Loss: 0.0119 Acc: 0.9974
val Loss: 0.0103 Acc: 0.9957

Epoch 8/24
----------
train Loss: 0.0078 Acc: 0.9982
val Loss: 0.0114 Acc: 0.9957

Epoch 9/24
----------
train Loss: 0.0054 Acc: 0.9988
val Loss: 0.0091 Acc: 0.9957

Epoch 10/24
----------
train Loss: 0.0042 Acc: 0.9996
val Loss: 0.0118 Acc: 0.9957

Epoch 11/24
----------
train Loss: 0.0035 Acc: 0.9994
val Loss: 0.0069 Acc: 0.9957

Ep

In [10]:
# Save trained model
torch.save(model_trained, 'vgg_19_fine_tune_whole_moire_classifier.pt')

In [3]:
# Load train model
model = torch.load('vgg_19_fine_tune_whole_moire_classifier.pt')

In [14]:
# Get classification report
get_classification_report(model, dataloaders, device)

'              precision    recall  f1-score   support\n\n       moire       0.98      0.99      0.99       330\n   non_moire       0.99      0.99      0.99       335\n\n    accuracy                           0.99       665\n   macro avg       0.99      0.99      0.99       665\nweighted avg       0.99      0.99      0.99       665\n'

In [5]:
# Get accuracy
test(model, dataloaders, device)


Test set: Accuracy: 656/665 (99%)

